# Chapter 21: Duality Theory and Practice

In [1]:
import numpy as np
from scipy import linalg as LA
import cvxpy as cp

## Circumscribed circle to a polytope relatively to the $\ell_\infty$-norm

The smallest $\ell_\infty$-ball containing the polytope $\mathcal{C} = \{x\in\mathbb{R}^d:Ax\leq b\}$, known as Chebyshev ball, has center $c^\star\in\mathbb{R}^d$ and radius $r^\star\in\mathbb{R}$ solving

$$ 
\mathop{\mathrm{minimize}}_{c\in\mathbb{R}^d,r\in\mathbb{R}} \qquad r \quad \mbox{ subject to } \|x-c\|_\infty \leq r \mbox{ for all } x\in\mathcal{C}. 
$$

This program can be transformed into a manageable linear program if the vertices $v_1,\dots,v_K$ of $\mathcal{C}$ are few and computable, since the constraint reads $\|v_k-c\|_\infty\leq r$ for all $k=1,\dots,K$. If this is not the case, the program can still be transformed into a manageable linear program using duality, as described in Example 21.4. 

For instance, when the rows of $A\in\mathbb{R}^{n\times d}$ are gaussian vectors normalized to have unit norm and when the $b\in\mathbb{R}^n$ is the vector of all ones, the polytope $\mathcal{C}$ becomes close to the unit euclidean ball as $n\to\infty$, so the Chebyshev radius should be close to (but larger than) one.

In [2]:
d = 5
n = 5000
A = np.random.randn(n,d)
for i in range(n):
    A[i,:] = A[i,:] / LA.norm(A[i,:])
b = np.ones((n,1))
c = cp.Variable((d,1))
r = cp.Variable(1)
Yp = cp.Variable((n,d),nonneg=True)
Ym = cp.Variable((n,d),nonneg=True)
objective = cp.Minimize(r)
constraints = [A.T@Yp==+np.identity(d)]
constraints+= [A.T@Ym==-np.identity(d)]
constraints+= [Yp.T@b-c<=r]
constraints+= [Ym.T@b+c<=r]
circle = cp.Problem(objective,constraints)
circle.solve(solver='ECOS')
print('The Chebyshev radius is {:.4f}. As expected, it is somewhat close to one.'.format(r.value[0]))

The Chebyshev radius is 1.0347. As expected, it is somewhat close to one.


# Owl-norm minimization for sparse recovery

In order to recover from $y=Ax\in\mathbb{R}^m$, $m\ll N$ , the sparse vectors $x\in\mathbb{R}^N$  whose entries corresponding to identical columns of $A$ are equal (not the sparest ones), the $\ell_1$-norm minimization can be replaced by

$$ 
\mathop{\mathrm{minimize} \;}_{z\in\mathbb{R}^N} \|z\|_{\mathrm{owl}} \quad \mbox{subject to} \quad Az=y. 
$$

As described in Example 21.5, this optimization program can be transformed into a linear program of manageable size.

In [3]:
# Create an observation matrix whose last two columns are identical...
N = 200
m = 100
A_aux = np.random.randn(m,N-1)
A = np.column_stack((A_aux,A_aux[:,N-2]))
# ...and a sparse vector with last two entries being equal
x = np.zeros(N)
s = 10
supp_aux = np.sort(np.random.permutation(N-2)[:s-1])
x[supp_aux] = np.random.randn(s-1)
x[N-1] = x[N-2] = 1/2
# produce the observation vector 
y = A@x

Note: to make sure of outputting a sparse solution, select a solver that will run the simplex algorithm.

In [4]:
# attempt to recover x from y by L1-norm minimization
from scipy.optimize import linprog
# the optimization variable is [xL1p;xL1m]
c = np.hstack( (np.ones(N), np.ones(N)) )   # the vector defining the objective function
A_eq = np.column_stack( (A,-A))             # the matrix involved in the equality constraint
b_eq = y                                    # the right-hand side of the equality constraint
bounds = [(0,None)]*(2*N)
# solving the linear program
res = linprog(c, A_eq=A_eq, b_eq=b_eq, method='revised simplex', bounds=bounds)
xL1p = res.x[:N]
xL1m = res.x[N:2*N]
xL1 = xL1p-xL1m
print('Recovery by L1-norm minimization is unsuccesful: the relative L2 error is {:.2e}.'.format(LA.norm(x-xL1)/LA.norm(x)))

Recovery by L1-norm minimization is unsuccesful: the relative L2 error is 3.66e-01.


In [5]:
# attempt to recover x from y by owl-norm minimization
import warnings
warnings.filterwarnings('ignore')
w = -np.sort(-np.random.rand(N))        # the weight vector is chosen arbitrarily
xOWL = cp.Variable(N)
a = cp.Variable(N)
b = cp.Variable(N)
objective = cp.Minimize(cp.sum(a)+cp.sum(b))
constraints = [A@xOWL == y]
constraints+= [ a[i]+b[j] >= +w[i]*xOWL[j] for i in range(N) for j in range(N)]
constraints+= [ a[i]+b[j] >= -w[i]*xOWL[j] for i in range(N) for j in range(N)]
OWL = cp.Problem(objective,constraints)
OWL.solve(solver='OSQP')
xOWL = xOWL.value
print('Recovery by owl-norm minimization is succesful: the relative L2 error is {:.2e}'.format(LA.norm(x-xOWL)/LA.norm(x)))

Recovery by owl-norm minimization is succesful: the relative L2 error is 1.70e-07


## Verification of semidefinite duality

Here, one simply checks that the semidefinite programs

$$ 
\mathop{\mathrm{minimize}}_{X\in\mathbb{R}^{d\times d}} \quad \mathop{\mathrm{tr}}(C^\top X) \quad \mbox{ subject to } \quad \mathop{\mathrm{tr}}(A_i^\top X), i=1,\dots,n \mbox{ and } X\succeq0
$$
and
$$ 
\mathop{\mathrm{maximize}}_{\nu\in\mathbb{R}^n} \quad \langle -b,\nu\rangle \quad \mbox{ subject to } \quad \nu_1 A_1 + \dots+\nu_n A_n + C\succeq0
$$

usually have the same optimal values.

In [6]:
d = 6
n = 10
# a loop appears to prevent cases of nonfeasibility of the primal program
primal_value = np.inf
while abs(primal_value) == np.inf:
    C = np.random.randn(d,d) 
    C = C+np.transpose(C)
    A = np.zeros((n,d,d))
    for i in range(n):
        aux = np.random.randn(d,d)
        A[i] = aux+np.transpose(aux) 
    b = np.random.rand(n)
    # the primal problem
    X = cp.Variable((d,d),PSD=True)
    objective = cp.Minimize(cp.trace(C.T@X))
    constraints = [cp.trace(A[i].T@X) == b[i] for i in range(n)] 
    primal = cp.Problem(objective,constraints)
    primal.solve()   
    primal_value = primal.value 
    # the dual problem
    nu = cp.Variable(n)    
    M = np.zeros((d,d))
    for i in range(n):
        M = M+nu[i]*A[i]
    objective = cp.Maximize(-b@nu)
    constraints = [(M+C)>>0] 
    dual = cp.Problem(objective,constraints)
    dual.solve()  
    dual_value = dual.value
print('The optimal values of the two problems agree: the primal value is {:.4f} and the dual value is {:.4f}.'.format(primal_value,dual_value))

The optimal values of the two problems agree: the primal value is -0.9966 and the dual value is -0.9966.
